##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
sys.path.append('../') # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [
    "pyyaml",
]

tools_to_install = [
    
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool
    


## 选择需要安装的模型

In [ ]:
from IPython.display import display,clear_output
from ipywidgets import Checkbox
import yaml
import os

new_yaml_path = './temp/selected_hf_models.yaml'

def create_folder(folder_name):
    try:
        os.makedirs(folder_name)
        print(f"{folder_name} created successfully")
    except:
        print(f"{folder_name} already exists")

folder_name = 'temp'
create_folder(folder_name)
clear_output()

def generate_new_yaml(file_path, selected_keys, new_file_path):
    with open(file_path, 'r') as file:
        yaml_data = yaml.safe_load(file)
    new_yaml = {}
    for key in selected_keys:
        new_yaml[key] = yaml_data[key]
        # 把recommended值都设置为True
        new_yaml[key]['recommended'] = True
    with open(new_file_path, 'w') as file:
        yaml.dump(new_yaml, file)

def display_yaml_checkboxes(file_path, default_checked_keys=[]):
    with open(file_path, 'r') as file:
        yaml_data = yaml.safe_load(file)
    checkboxes = []
    selected_keys = []
    def on_value_change(change):
        if change['new']:
            selected_keys.append(change['owner'].description)
        else:
            selected_keys.remove(change['owner'].description)
        generate_new_yaml(file_path, selected_keys, new_yaml_path)
    for key in yaml_data.keys():
        checkbox = Checkbox(description=key, value=(key in default_checked_keys))
        checkbox.observe(on_value_change, 'value')
        checkboxes.append(checkbox)
        display(checkbox)
    # display(*checkboxes)
    selected_keys += default_checked_keys
    generate_new_yaml(file_path, selected_keys, new_yaml_path)
    return selected_keys

file_path = './hf_models_collection.yaml'

default_checked_keys = [
    "anything-v4.0",
]
display("请挑选需要安装的模型")
selected_keys = display_yaml_checkboxes(file_path, default_checked_keys)